In [145]:
import numpy as np
import os

In [146]:
### Please note that code in this notebook is as stupid as an African penguin doing quantum physics. Thanks :)

In [279]:
def read_model_file(filename):
    coords = np.empty((0, 3))
    with open(filename) as f:
        line = f.readline()
        model_to_coords = {}
        current_model = -1
        while line:
            words = line.strip().split(' ')
            if len(words) > 0:
                if words[0] == "MODEL":
                    current_model = words[1]
#                     print("Current models is: MODEL", current_model)
                elif words[0] == "ATOM":
                    if current_model == -1:
                        raise NameError('Unkown model with MODEL={current_model}')
                    x, y, z = 0, 0, 0
                    coord_starts = [6, 7, 8]
                    last_nonempty = 0
                    for idx, word in enumerate(words):
                        if len(word.strip()) > 0:
                            last_nonempty += 1
                            if last_nonempty == coord_starts[0]:
                                x = word
                            elif last_nonempty == coord_starts[1]:
                                y = word
                            elif last_nonempty == coord_starts[2]:
                                z = word
                    current_model_coords = np.array([[x, y, z]])
#                     print(f'Atom coords for MODEL={current_model} is {current_model_coords}')
                    coords = np.append(coords, current_model_coords, axis=0)
                elif words[0] == "ENDMDL":
                    model_to_coords[int(current_model)] = coords.astype(np.float64)
#                     print(f'Finished MODEL={current_model}')
                    current_model = -1
            line = f.readline()
    return model_to_coords

In [280]:
def read_reslog_file(filename):
    model_to_use = []
    with open(filename) as f:
        line = f.readline()
        while line:
            words = [w for w in line.strip().split(' ') if w]
            if len(words) == 4:
                if float(words[2]) <= 2.0:
                    model_to_use.append(int(words[0]))
            line = f.readline()
    return model_to_use

In [281]:
def calculate_model_mean(model):
    return np.mean(model, axis=0, dtype=np.float64)

In [303]:
def calculate_centers(rootdir):
#     centers = np.empty((0, 3))
    centers = []
    dictfilt = lambda x, y: dict([ (i,x[i]) for i in x if i in set(y) ])
    for subdir, dirs, filenames in os.walk(rootdir):
        if subdir == rootdir:
            continue
            
        print("===")
        print(f'Current directory is "{subdir}"')
        reslog = os.path.join(subdir, [f for f in filenames if f.endswith(".log")][0])
        pdbqt = os.path.join(subdir, [f for f in filenames if f.endswith(".pdbqt")][0])
        print(f'Current reslog file is "{reslog}"')
        print(f'Current model file is "{pdbqt}"')
        model_keys_to_use = read_reslog_file(reslog)
        print(model_keys_to_use)
        models_all = read_model_file(pdbqt)
        models_to_use = dictfilt(models_all, model_keys_to_use)
        print(models_to_use.keys())
#         print(f'Models to use {model_keys_to_use}')
        for model in models_to_use.values():
#             print(f'model is {model}')
#             np.append(centers, calculate_model_mean(model), axis=0)
            centers.append(calculate_model_mean(model))
    print (centers)


In [304]:
centers = calculate_centers("./data/")

===
Current directory is "./data/3"
Current reslog file is "./data/3/res.log"
Current model file is "./data/3/Str-2 3D_out.pdbqt"
[1, 2, 9]
dict_keys([1, 2, 9])
===
Current directory is "./data/2"
Current reslog file is "./data/2/res.log"
Current model file is "./data/2/Str-2 3D_out.pdbqt"
[1, 8]
dict_keys([1, 8])
===
Current directory is "./data/1"
Current reslog file is "./data/1/res.log"
Current model file is "./data/1/Str-2_3D_out.pdbqt"
[1, 5, 9]
dict_keys([1, 5, 9])
===
Current directory is "./data/5"
Current reslog file is "./data/5/res.log"
Current model file is "./data/5/Str-2 3D_out.pdbqt"
[1, 2, 5, 9]
dict_keys([1, 2, 5, 9])
===
Current directory is "./data/4"
Current reslog file is "./data/4/res.log"
Current model file is "./data/4/Str-2 3D_out.pdbqt"
[1, 2, 5, 9]
dict_keys([1, 2, 5, 9])
[array([ 6.94495833,  0.92654167, 79.82533333]), array([ 6.81447917,  0.864125  , 80.26052083]), array([-1.42072685,  5.14905093, 68.67189352]), array([ 6.94654167,  0.92225   , 79.82358333